In [6]:
import arcpy
import os 
inDir = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer"
fluxDir = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\Luwen_Nutrient"
inRasterTN = os.path.join(fluxDir,"TN_Annual_delTotal_header_kgcellday.tif")
inRasterTP = os.path.join(fluxDir,"TP_Annual_delTotal_header_kgcellday.tif")
Huronbasin = os.path.join(inDir, "greatlakes_subbasins", "Huronbasin.shp")
outDir = fluxDir
arcpy.env.workspace = outDir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3174)

In [7]:
# ---- Prep: make a clean, single-part polygon mask that matches raster CRS ----
def prep_mask_for_raster(raster_path, mask_fc):
    """Returns a temporary polygon feature class projected to the raster's CRS and dissolved."""
    ras_sr = arcpy.Describe(raster_path).spatialReference
    tmp_gdb = arcpy.management.CreateFileGDB(outDir, "tmp_clip_gdb.gdb")[0] \
             if not arcpy.Exists(os.path.join(outDir, "tmp_clip_gdb.gdb")) else os.path.join(outDir, "tmp_clip_gdb.gdb")
    mask_diss = os.path.join(tmp_gdb, "mask_diss")
    mask_proj = os.path.join(tmp_gdb, "mask_proj")

    # 1) Repair geometry (saves headaches)
    arcpy.management.RepairGeometry(mask_fc)

    # 2) Dissolve to a single polygon (handles multipart/overlaps)
    if not arcpy.Exists(mask_diss):
        arcpy.management.Dissolve(mask_fc, mask_diss)

    # 3) Project mask to raster CRS if needed
    if ras_sr.name and ras_sr.factoryCode not in (0, None):
        arcpy.management.Project(mask_diss, mask_proj, ras_sr)
        return mask_proj
    else:
        # If raster SR missing, just use dissolved mask
        return mask_diss

# ---- Clip via Data Management > Clip (uses polygon shape) ----
def clip_raster_to_mask(in_raster, mask_fc, out_path):
    # Align pixels to the input raster
    arcpy.env.snapRaster = in_raster
    arcpy.env.cellSize = in_raster

    # Prepare a polygon mask that matches the raster CRS
    mask_for_this_raster = prep_mask_for_raster(in_raster, mask_fc)

    # Run Clip (Data Management) with ClippingGeometry so it trims to polygon, not extent
    # rectangle="#": use mask extent; MAINTAIN_EXTENT keeps exact raster grid alignment.
    arcpy.management.Clip(
        in_raster=in_raster,
        rectangle="#",
        out_raster=out_path,
        in_template_dataset=mask_for_this_raster,
        nodata_value="NoData",
        clipping_geometry="ClippingGeometry",
        maintain_clipping_extent="MAINTAIN_EXTENT"
    )
    print(f"✔ Clipped: {os.path.basename(in_raster)} -> {out_path}")

# ---- Run for both rasters ----
tn_out = os.path.join(outDir, "TN_Annual_delTotal_header_kgcellday_Huron.tif")
tp_out = os.path.join(outDir, "TP_Annual_delTotal_header_kgcellday_Huron.tif")

clip_raster_to_mask(inRasterTN, Huronbasin, tn_out)
clip_raster_to_mask(inRasterTP, Huronbasin, tp_out)

✔ Clipped: TN_Annual_delTotal_header_kgcellday.tif -> D:\Users\abolmaal\Arcgis\NASAOceanProject\Luwen_Nutrient\TN_Annual_delTotal_header_kgcellday_Huron.tif
✔ Clipped: TP_Annual_delTotal_header_kgcellday.tif -> D:\Users\abolmaal\Arcgis\NASAOceanProject\Luwen_Nutrient\TP_Annual_delTotal_header_kgcellday_Huron.tif


In [2]:
import numpy as np

def quantiles(data, probs=(0, 0.025, 0.05, 0.075, 1.0), log_space=False):
    """
    data: 1D array-like of values
    probs: tuple/list of probabilities in [0,1]
    log_space: if True, compute quantiles in log10 space (good for wide ranges)
    """
    x = np.asarray(data, dtype=float)
    x = x[np.isfinite(x)]  # drop NaN/inf

    if not len(x):
        raise ValueError("No finite values in data.")

    if log_space:
        # require positive values to take log10
        x_pos = x[x > 0]
        if not len(x_pos):
            raise ValueError("All values are <= 0; cannot compute log-space quantiles.")
        q_log = np.quantile(np.log10(x_pos), probs)
        return 10 ** q_log
    else:
        return np.quantile(x, probs)

# EXAMPLE (replace with your real array)
vals = [30.74, 1.713e-10, 0.002, 0.5, 3.4, 12.1]  # your full data goes here
qs_lin = quantiles(vals, probs=np.linspace(0, 1, 6))          # 0%,20%,...,100%
qs_log = quantiles(vals, probs=np.linspace(0, 1, 6), log_space=True)
print("Linear-space quantiles:", qs_lin)
print("Log-space quantiles:   ", qs_log)

Linear-space quantiles: [1.713e-10 2.000e-03 5.000e-01 3.400e+00 1.210e+01 3.074e+01]
Log-space quantiles:    [1.713e-10 2.000e-03 5.000e-01 3.400e+00 1.210e+01 3.074e+01]


In [40]:
import numpy as np


import numpy as np

def six_quantiles_in_range_precise(
    data,
    vmin=0.00015,
    vmax=0.0009,
    log_space=True,
    method="hazen",          # 'weibull', 'hazen', 'median_unbiased', etc. (NumPy>=1.22)
    on_empty="clip",         # 'clip' or 'error'
    ensure_strict=True       # nudge ties to make strictly increasing
):
    """
    Return 6 quantiles (0%,20%,40%,60%,80%,100%) from values within [vmin, vmax].
    - log-space quantiles for wide ranges (recommended)
    - 'method' controls the quantile estimator (better small-sample behavior)
    - endpoints are pinned to vmin/vmax
    """
    x_all = np.asarray(data, dtype=float)
    x_all = x_all[np.isfinite(x_all)]
    if x_all.size == 0:
        raise ValueError("No finite values in data.")

    # restrict to range (or clip if empty)
    mask = (x_all >= vmin) & (x_all <= vmax)
    x = x_all[mask]
    if x.size == 0:
        if on_empty == "error":
            raise ValueError("No data points within [vmin, vmax].")
        x = np.clip(x_all, vmin, vmax)

    probs = np.linspace(0, 1, 6)

    if log_space:
        # safety: positive only
        x = x[x > 0]
        if x.size == 0:
            raise ValueError("No positive values in range for log-space quantiles.")
        q = 10 ** np.quantile(np.log10(x), probs, method=method)
    else:
        q = np.quantile(x, probs, method=method)

    # pin endpoints exactly to the requested bounds
    q[0] = vmin
    q[-1] = vmax

    # enforce strict monotonicity if ties happen
    if ensure_strict:
        for i in range(1, len(q)):
            if q[i] <= q[i-1]:
                q[i] = np.nextafter(q[i-1], np.inf)

    return q

# --- usage ---
q_P = six_quantiles_in_range_precise(vals, vmin=0.000015, vmax=0.1, log_space=True, method="hazen")
print(q_P)

[1.5e-05 2.0e-03 2.0e-03 2.0e-03 2.0e-03 1.0e-01]


In [ ]:
q_P= six_quantiles_in_range(vals, vmin=0.00015, vmax=15, log_space=True)
print(q_P)

In-range count: 0 / 6  (vmin=0.00015, vmax=0.0009)
[0.00015 0.0009  0.0009  0.0009  0.0009  0.0009 ]
